In [66]:
import numpy as np
import pandas as pd
from scrapy.selector import Selector
from selenium import webdriver 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm
import warnings
import re

import os
import requests
from PIL import Image
from io import BytesIO
from urllib.parse import urlparse
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

In [67]:
# Instantiate the data required in lists
users_list = ['ur22171966']
#'ur22171966' 1k+
#ur46592925 4 reviews
user_id = []
title_list = []
description_list = []
img_list = []
year_list = []
director_list = []
star_list = []
duration_list = []
advisory_list = []
genre_list = []
vote_list = []
movie_rating_list = []
user_rating_list = []
img_file_list = []
error_msg = []

In [68]:
for user in range(len(users_list)):
    print("Scraping for User {}".format(user+1))
    driver = webdriver.Chrome(ChromeDriverManager().install())
    url = 'https://www.imdb.com/user/{}/ratings'.format(users_list[user])
    time.sleep(1)
    driver.get(url)
    time.sleep(1)
    
    sel = Selector(text = driver.page_source)
    num_of_ratings = sel.css(".lister-list-length span::text").extract_first().replace(',','').split(' ')[0]
    rating_pages = int(int(num_of_ratings)/100) + 1  
    user_id += [users_list[user] for i in range(int(num_of_ratings))]
    
    #scroll
    scroll_height = driver.execute_script("return document.body.scrollHeight")
    
    # Set scroll step and delay
    scroll_step = 700 
    scroll_delay = 1  

    current_scroll_position = 0

    for x in range(rating_pages):
        sel = Selector(text = driver.page_source)
        reviews = driver.find_elements(By.CSS_SELECTOR, 'div.lister-item.mode-detail')
        
        current_scroll_position = 0
        scroll_height = driver.execute_script("return document.body.scrollHeight")
        while current_scroll_position < scroll_height:
            driver.execute_script(f"window.scrollTo(0, {current_scroll_position + scroll_step});")
            current_scroll_position += scroll_step
            time.sleep(scroll_delay)
            new_scroll_height = driver.execute_script("return document.body.scrollHeight")
            if new_scroll_height > scroll_height:
                scroll_height = new_scroll_height
            if current_scroll_position >= scroll_height:
                break
        
        for review in tqdm(reviews):
            try:
                sel2 = Selector(text = review.get_attribute('innerHTML'))
                
                ##Get img path
                try:
                    img_path = sel2.css('.lister-item-image.ribbonize img::attr(src)').extract_first().strip()
                except:
                    img_path = np.NaN
                
                ## Get movie title
                try:
                    title = sel2.css('.lister-item-content a::text').extract_first().strip()
                    episode = sel2.css('.lister-item-content a::text').getall()[1].strip()
                    if episode != "":
                        title += (" - " + episode)
                except:
                    title= np.NaN
                    
                ## Get movie description
                try:
                    advisory = sel2.css('.certificate::text').extract_first()
                    duration = sel2.css('.runtime::text').extract_first()
                    description = None
                    if advisory == None and duration == None:
                        description = sel2.css('p::text').getall()[3].strip()
                    elif advisory == None or duration == None:
                        description = sel2.css('p::text').getall()[5].strip()
                    else:
                        description = sel2.css('p::text').getall()[7].strip()
                except:
                    description = np.NaN
                ## Get movie year
                try:
                    year = sel2.css('.lister-item-year.text-muted.unbold::text').extract_first().strip().replace('(','').replace(')','')
                    year = re.sub(r'[a-zA-Z\s]+', '', year)
                except:
                    year = np.NaN
                ## Get directors and staff
                try:
                    staff = sel2.css('.text-muted a::text').getall()
                    text = sel2.css('.text-muted.text-small::text').getall()
                    text2 = [x.strip() for x in text]
                    commas = text2.count(',')
                    stars_index = text2.index("Stars:")
                    count = 0
                    for i in range(stars_index, len(text2)-1):
                        if text2[i] == ',':
                            count+=1
                    stars = staff[-(count+1):]
                    # if directors are recorded
                    if "Director:" in text2 or "Directors:" in text2:
                        directors = staff[:(commas-count)+1]
                    else:
                        directors = ""
                except:
                    stars = np.NaN
                    directors = np.NaN
                ## Get movie duration
                try:
                    duration = duration.strip()
                except:
                    duration = np.NaN
                ## Get viewer advisory
                try:
                    advisory = advisory.strip()
                except:
                    advisory = np.NaN
                ## Get Genre
                try:
                    genre = sel2.css('.genre::text').extract_first().strip()
                except:
                    genre = np.NaN
                ## Get votes
                try:
                    votes = sel2.css('.text-muted.text-small span::text').getall()[-1]
                    votes = int(votes.replace(',','').split(' ')[0])
                except:
                    votes = np.NaN
                ## Get movie rating
                try:
                    movie_rating = sel2.css('.ipl-rating-star__rating::text').getall()[0]
                    movie_rating = float(movie_rating.replace(',','').split(' ')[0])
                except:
                    movie_rating = np.NaN
                ## Get user rating
                try:
                    user_rating = sel2.css('.ipl-rating-star__rating::text').getall()[1]
                    user_rating = int(user_rating.replace(',','').split(' ')[0])
                except:
                    user_rating = np.NaN
                    
                #image file name
                try:
                    title_rename = re.sub(r'[^\w\s!-]|[.!?]', '', title)
                    img_name = f"{user_id[0]}_{title_rename}.jpg"
                except:
                    img_name = np.NaN
                
                title_list.append(title)
                img_list.append(img_path)
                description_list.append(description)
                year_list.append(year)
                director_list.append(directors)
                star_list.append(stars)
                duration_list.append(duration)
                advisory_list.append(advisory)
                genre_list.append(genre)
                vote_list.append(votes)
                movie_rating_list.append(movie_rating)
                user_rating_list.append(user_rating)
                img_file_list.append(img_name)
                
            except Exception as e:
                error_msg.append(e)
        
        try:
            next_page_url = sel.css("a.flat-button.lister-page-next.next-page::attr(href)").extract_first()
            driver.get("https://www.imdb.com" + next_page_url)
            
            
        except:
            print("No more pages to browse")

Scraping for User 1


100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 102.90it/s]

No more pages to browse


In [69]:
# Storing all data in dataframe
rating_df = pd.DataFrame({
     "UserID": user_id,
     "Title":title_list,
     "Img_Path": img_list,
     "Img_File_Name": img_file_list,
     "Year":year_list,
     "Description":description_list,
     "Directors":director_list,
     "Stars": star_list,
     "Viewer_Advisory": advisory_list,
     "Duration": duration_list,
     "Genre": genre_list,
     "Votes": vote_list, 
     "Movie_Rating": movie_rating_list,
     "User_Rating": user_rating_list})

In [70]:
# Export dataset
rating_df.to_csv(path_or_buf = f"{user_id[0]}_ratings.csv"
                          , index = False,  encoding='utf-8-sig')

In [73]:
##image downloading
def download_image(url, save_path):
    try:
        response = requests.get(url)
        response.raise_for_status()

        img = Image.open(BytesIO(response.content))


        img_save_path = save_path
        img.save(img_save_path, format="JPEG")

        print(f"Image saved at {img_save_path}")
    except Exception as e:
        print(f"Error downloading image from {url}: {e}")

# Folder for downloading
img_folder = f"{user_id[0]}_downloaded_images"
os.makedirs(img_folder, exist_ok=True)

for idx, row in rating_df.iterrows():
    user_id = row['UserID']
    title = row['Title']
    img_url = row['Img_Path']
    
    title = re.sub(r'[^\w\s!-]|[.!?]', '', title)

    # File extension name
    img_filename = f"{user_id}_{title}.jpg"
    img_save_path = os.path.join(img_folder, img_filename)

    # Download and save the image
    download_image(img_url, img_save_path)
    
    #See image in python
    #img = Image.open(img_save_path)
    #plt.imshow(img)
    #plt.savefig(img_save_path)
    #plt.show()

Image saved at ur22171966_downloaded_images\ur22171966_Goosebumps.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Seoul Station.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Hacksaw Ridge.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Infernal Affairs III.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Infernal Affairs II.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Infernal Affairs.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Moonlight.jpg
Image saved at ur22171966_downloaded_images\ur22171966_La La Land.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Arrival.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Nikita.jpg
Image saved at ur22171966_downloaded_images\ur22171966_The Eyes of My Mother.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Blade Runner.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Kiss Kiss Bang Bang.jpg
Image saved at ur22171966_downloaded_images\u

Image saved at ur22171966_downloaded_images\ur22171966_Ghost World.jpg
Image saved at ur22171966_downloaded_images\ur22171966_The Nice Guys.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Green Room.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Bone Tomahawk.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Spotlight.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Hardcore Henry.jpg
Image saved at ur22171966_downloaded_images\ur22171966_The Usual Suspects.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Reservoir Dogs.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Amadeus.jpg
Image saved at ur22171966_downloaded_images\ur22171966_The Good Dinosaur.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Batman v Superman Dawn of Justice.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Eat.jpg
Image saved at ur22171966_downloaded_images\ur22171966_They Look Like People.jpg
Image saved at ur22171966_downloaded_

Image saved at ur22171966_downloaded_images\ur22171966_Barely Lethal.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Four Lions.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Batman Bad Blood.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Zodiac.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Total Recall.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Total Recall.jpg
Image saved at ur22171966_downloaded_images\ur22171966_The Truman Show.jpg
Image saved at ur22171966_downloaded_images\ur22171966_The Girl with the Dragon Tattoo.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Valkyrie.jpg
Image saved at ur22171966_downloaded_images\ur22171966_The Drop.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Se7en.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Harry Brown.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Captive.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Spring.jpg


Image saved at ur22171966_downloaded_images\ur22171966_Detective K Secret of Virtuous Widow.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Kill Me Three Times.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Sex Ed.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Cop Car.jpg
Image saved at ur22171966_downloaded_images\ur22171966_The Sixth Sense.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Memories of the Sword.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Collateral.jpg
Image saved at ur22171966_downloaded_images\ur22171966_The Voices.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Big Eyes.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Whiplash.jpg
Image saved at ur22171966_downloaded_images\ur22171966_RIPD.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Star Wars Episode II - Attack of the Clones.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Lost River.jpg
Image saved at ur221719

Image saved at ur22171966_downloaded_images\ur22171966_Gone Girl.jpg
Image saved at ur22171966_downloaded_images\ur22171966_The Boxtrolls.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Tazza The Hidden Card.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Big Hero 6.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Pride.jpg
Image saved at ur22171966_downloaded_images\ur22171966_The November Man.jpg
Image saved at ur22171966_downloaded_images\ur22171966_The Two Faces of January.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Before I Go to Sleep.jpg
Image saved at ur22171966_downloaded_images\ur22171966_The Guest.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Horrible Bosses 2.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Lets Be Cops.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Dumb and Dumber To.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Dumb and Dumber.jpg
Image saved at ur22171966_downl

...

Image saved at ur22171966_downloaded_images\ur22171966_Toy Story 2.jpg
Image saved at ur22171966_downloaded_images\ur22171966_The Pianist.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Toy Story.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Buffalo 66.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Ninja Assassin.jpg
Image saved at ur22171966_downloaded_images\ur22171966_When Harry Met Sally.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Ninja.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Undisputed 3 Redemption.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Undisputed.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Survival of the Dead.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Undisputed 2 Last Man Standing.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Brooklyns Finest.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Diary of the Dead.jpg
Image saved at ur2217196

Image saved at ur22171966_downloaded_images\ur22171966_Devil.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Zombie Diaries.jpg
Image saved at ur22171966_downloaded_images\ur22171966_District B13.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Grindhouse.jpg
Image saved at ur22171966_downloaded_images\ur22171966_An American Crime.jpg
Image saved at ur22171966_downloaded_images\ur22171966_The Girl Next Door.jpg
Image saved at ur22171966_downloaded_images\ur22171966_District 13 Ultimatum.jpg
Image saved at ur22171966_downloaded_images\ur22171966_I Give My First Love to You.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Crush and Blush.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Easy A.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Fame.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Mean Girls.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Vacancy.jpg
Image saved at ur22171966_downloaded_images\ur22171966

Image saved at ur22171966_downloaded_images\ur22171966_The Accidental Gangster and the Mistaken Courtesan.jpg
Image saved at ur22171966_downloaded_images\ur22171966_17 Again.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Run Fatboy Run.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Dead Wood.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Inkheart.jpg
Image saved at ur22171966_downloaded_images\ur22171966_The Invention of Lying.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Ponyo.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Hotel Rwanda.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Zombie Strippers.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Pulp Fiction.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Big Man Japan.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Franklyn.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Road to Perdition.jpg
Image saved at ur22171966_

Image saved at ur22171966_downloaded_images\ur22171966_Dance Flick.jpg
Image saved at ur22171966_downloaded_images\ur22171966_The Men Who Stare at Goats.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Delta Farce.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Bride Wars.jpg
Image saved at ur22171966_downloaded_images\ur22171966_The Rocker.jpg
Image saved at ur22171966_downloaded_images\ur22171966_May.jpg
Image saved at ur22171966_downloaded_images\ur22171966_The Box.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Wrong Turn 3 Left for Dead.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Sorority Row.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Gacy.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Saw IV.jpg
Image saved at ur22171966_downloaded_images\ur22171966_Role Models.jpg
Image saved at ur22171966_downloaded_images\ur22171966_The Taking of Pelham One Two Three.jpg
Image saved at ur22171966_downloaded_images\ur221

Image saved at ur22171966_downloaded_images\ur22171966_Alien.jpg
